# DS PROJECT - Predicting the outcome of driving exams in Estonia

## Reading and examining the data

### Reading and formatting

In [61]:
import pandas as pd

# Read the three CSV files into Pandas DataFrames

data2021 = pd.read_csv("se_2021.csv")
data2022 = pd.read_csv("se_2022.csv")
data2023 = pd.read_csv("se_2023.csv")

# Combine the three DataFrames
combined_data = pd.concat([data2021, data2022, data2023], axis=0, ignore_index=True)

#split KUUPAEV into AASTA and KUU
combined_data[['AASTA', 'KUU']] = combined_data['KUUPAEV'].str.split('-', expand=True)
combined_data['AASTA'] = pd.to_numeric(combined_data['AASTA'])
combined_data['KUU'] = pd.to_numeric(combined_data['KUU'])

#drop KUUPAEV
combined_data = combined_data.drop(columns=['KUUPAEV'])
combined_copy = combined_data.copy()

combined_data.head()

,EKSAMI_SOORITAJA,BYROO,KATEGOORIA,ERITINGIMUSED,VIIMANE_AUTOKOOL,SOIDUOPETAJA_KAASAS,EKSAMINEERIJA,SEISUND,KESTUS,KATK_POHJUS,MITTEARVESTATUD,VEAD,AASTA,KUU
0,fd4a476c2fff5845,Haapsalu,B,101,osaühing KLIMBERG,E,31451a368e962058,SOORITATUD,50.0,NaN,NaN,NaN,2021,1
1,8fe3816987e62971,Jõgeva,B,101,OÜ AUTOSÕIT,E,ae264be409f2ca9f,MITTE_SOORITATUD,24.0,NaN,EESOIGUS,NaN,2021,1
2,1131ad82d5710721,Põlva,B,101,H&M MERTENS OÜ,E,40bcf2cea820fca6,MITTE_SOORITATUD,24.0,NaN,EESOIGUS,NaN,2021,1
3,819bfde54073d8e7,Võru,B,101,Tugev Partner OÜ,E,bf2b7218d36e2a3d,MITTE_SOORITATUD,48.0,NaN,JALGIMINE|LIIKLUSOHUTUS|TEELPUSIMINE,NaN,2021,1
4,06493201c082d7f2,Haapsalu,B,101,OÜ Raak Company,E,31451a368e962058,SOORITATUD,51.0,NaN,NaN,NaN,2021,1


### Looking at values in the columns and encoding categorical variables into numerical representations

In [62]:
combined_data['BYROO'].value_counts(dropna=False)

Tallinn       49056
Tartu         15323
Rakvere       10772
Pärnu          9759
Jõhvi          8623
Narva          6262
Paide          6041
Viljandi       5306
Rapla          4097
Haapsalu       3709
Võru           3446
Kuressaare     3232
Valga          1052
Jõgeva          912
Põlva           907
Kärdla          718
Saue              2
Name: BYROO, dtype: int64

In [ ]:
#one-hot

In [63]:
combined_data['KATEGOORIA'].value_counts(dropna=False)

B      91552
C       9684
A       8310
CE      7225
BE      3834
T       2599
D       1920
AM      1678
A2      1377
A1       836
C1       175
Z         22
C1E        5
Name: KATEGOORIA, dtype: int64

In [ ]:
#one-hot

In [64]:
combined_data['ERITINGIMUSED'].value_counts(dropna=False) #uuri 101|78|78, kui ei saa selgust muuda 101|78-ks, neid ainult 3

101          69243
NaN          37012
101|78       19195
79.01         1331
96            1091
78             982
79.02          347
106             13
101|78|78        3
Name: ERITINGIMUSED, dtype: int64

In [65]:
combined_copy['ERITINGIMUSED'] = combined_copy['ERITINGIMUSED'].replace({'101|78|78': '101|78'})
#one-hot

In [66]:
combined_data['VIIMANE_AUTOKOOL'].value_counts(dropna=False).sort_index() 

"Autom" Põltsamaa Liiklusklubi     225
A&J AUS OÜ                           9
A+ AUTOKOOL OÜ                     256
A-AUTOM, OÜ                         23
A2 autokool OÜ                      27
                                  ... 
osaühing THULEMANN                   4
osaühing VASARA AUTOKOOL           628
osaühing Valga Makd                 28
Õnneleid Liikluskool OÜ            444
NaN                               7866
Name: VIIMANE_AUTOKOOL, Length: 341, dtype: int64

In [ ]:
#vaata sarnased ja lühendid läbi, kõik variandid osaühingutest jms lühenditeks nagu OÜ, one-hot

In [67]:
combined_data['SOIDUOPETAJA_KAASAS'].value_counts(dropna=False)

E      119935
J        5758
NaN      3524
Name: SOIDUOPETAJA_KAASAS, dtype: int64

In [84]:
#Nanid 0-ks, binaarne
import numpy as np
combined_copy['SOIDUOPETAJA_KAASAS'] = combined_copy['SOIDUOPETAJA_KAASAS'].replace({'E': 0, 'J': 1, np.nan: 0})
combined_copy['SOIDUOPETAJA_KAASAS'] = combined_copy['SOIDUOPETAJA_KAASAS'].astype(int)

In [69]:
combined_data['EKSAMI_SOORITAJA'].value_counts(dropna=False)

7da809d9d7b050c3    18
293e4a0f5ded2650    17
8f5fc6fb5d3f10a2    16
9389bc6e7479bc10    16
2e5a49e327ad8546    16
                    ..
ee1f85ae4d8e7d75     1
9727e3ffd56f6da2     1
b721a3b4b7e5f826     1
836ec1b5c8bb5727     1
32426dfae15e014f     1
Name: EKSAMI_SOORITAJA, Length: 68303, dtype: int64

In [ ]:
#new column for how many times has taken exam (think through how to implement), after drop eksami_sooritaja

In [70]:
combined_data[['EKSAMINEERIJA','AASTA']].value_counts(dropna=False).sort_index()

EKSAMINEERIJA     AASTA
00fe4a180f5c474e  2021       40
                  2023        2
075e15cf091d347b  2021     1143
                  2022     1225
                  2023      897
                           ... 
faeaf99580d6be30  2022     1177
                  2023      448
ff4d9b9b519322e3  2021     1277
                  2022     1261
                  2023      979
Length: 133, dtype: int64

In [ ]:
#one-hot

In [71]:
combined_data['SEISUND'].value_counts(dropna=False)

SOORITATUD           70087
MITTE_SOORITATUD     55298
EI_ILMUNUD_KOHALE     2604
KATKESTATUD           1228
Name: SEISUND, dtype: int64

In [72]:
#muuda numbriteks
combined_copy['SEISUND'] = combined_copy['SEISUND'].replace({'SOORITATUD': 1, 'MITTE_SOORITATUD': 0})
combined_copy['SEISUND'] = combined_copy['SEISUND'].astype(int)

In [73]:
combined_data['KESTUS'].describe()

count    125414.000000
mean         39.512088
std          18.024153
min           0.000000
25%          27.000000
50%          43.000000
75%          51.000000
max        1024.000000
Name: KESTUS, dtype: float64

In [15]:
combined_data[combined_data['KESTUS']>200].sort_values(by='KESTUS', ascending=False)


,EKSAMI_SOORITAJA,BYROO,KATEGOORIA,ERITINGIMUSED,VIIMANE_AUTOKOOL,SOIDUOPETAJA_KAASAS,EKSAMINEERIJA,SEISUND,KESTUS,KATK_POHJUS,MITTEARVESTATUD,VEAD,AASTA,KUU
25174,ec13762fd80284e5,Pärnu,B,101,OÜ AUTOSÕIT,E,c347692cf48342d6,SOORITATUD,1024.0,NaN,NaN,NaN,2021,9
119155,8ce269441d03d7f0,Tallinn,CE,NaN,Ametikoolitus OÜ,E,5c9ee299be483144,MITTE_SOORITATUD,1001.0,NaN,TAGUR_BOKS,EBAPIISAV_ETTEVALMISTUS|VASTU_KOONUST,2023,4
33985,b3b1e42554e36eef,Tallinn,C,NaN,OÜ AUTOSÕIT,E,d15d81b55baf64ed,SOORITATUD,947.0,NaN,NaN,NaN,2021,10
88657,2f2fe3e872882aa5,Tallinn,C,NaN,Stalker Autokool Osaühing,J,eb13365f3504e035,SOORITATUD,473.0,NaN,NaN,NaN,2022,9
89040,8d31248048c2b794,Rakvere,C,NaN,Mittetulundusühing Viljandi Autom,E,075e15cf091d347b,SOORITATUD,450.0,NaN,NaN,NaN,2022,11
126344,354894192d080741,Tallinn,D,NaN,Aktsiaselts Tallinna Linnatransport,E,4b9e369e12d8f969,MITTE_SOORITATUD,380.0,NaN,ISESEISVUS,NaN,2023,6
31824,ebcc3e04a8b0c077,Võru,C,NaN,Tugev Partner OÜ,E,faeaf99580d6be30,SOORITATUD,376.0,NaN,NaN,NaN,2021,10
102461,ecdcf1490764bc3a,Jõgeva,B,101,VPK Koolitus OÜ,E,b89f5e763b05e7a7,SOORITATUD,373.0,NaN,NaN,NaN,2023,5
119565,fa2cfcee7f82cb4f,Tallinn,D,NaN,Aktsiaselts Tallinna Linnatransport,J,4ee4a4702131dac3,SOORITATUD,371.0,NaN,NaN,NaN,2023,1
122541,54db071c7a3ad3d3,Rakvere,C,NaN,Osaühing TAPA AUTOKOOL,E,4141299ff500eb11,SOORITATUD,356.0,NaN,NaN,NaN,2023,10


In [ ]:
#drop liiga suured ja NaNid 0.0-iks (pole tegelikult vaja)

In [74]:
combined_data['KATK_POHJUS'].value_counts(dropna=False)

NaN            125406
EI_ILMUNUD       2532
MUUD              683
TEHN_RIKE         202
KEHTETU_DOK       187
ILM_TEEOLUD       142
TERVIS             49
ONNETUS            16
Name: KATK_POHJUS, dtype: int64

In [75]:
pd.set_option('display.max_rows', 20)
combined_data[(combined_data['KATK_POHJUS']=='EI_ILMUNUD')|(combined_data['SEISUND']=='EI_ILMUNUD_KOHALE')].sort_values(by='SEISUND')

,EKSAMI_SOORITAJA,BYROO,KATEGOORIA,ERITINGIMUSED,VIIMANE_AUTOKOOL,SOIDUOPETAJA_KAASAS,EKSAMINEERIJA,SEISUND,KESTUS,KATK_POHJUS,MITTEARVESTATUD,VEAD,AASTA,KUU
25,f73f8ee69d949fe4,Viljandi,B,101,Olustvere Teenindus- ja Maamajanduskool,NaN,c68c6310b2fe1124,EI_ILMUNUD_KOHALE,NaN,EI_ILMUNUD,NaN,NaN,2021,1
84235,024c781269865394,Tallinn,CE,NaN,Ametikoolitus OÜ,NaN,c68c6310b2fe1124,EI_ILMUNUD_KOHALE,NaN,EI_ILMUNUD,NaN,NaN,2022,1
84288,4de3200319b92823,Pärnu,B,NaN,NaN,NaN,c68c6310b2fe1124,EI_ILMUNUD_KOHALE,NaN,EI_ILMUNUD,NaN,NaN,2022,6
84333,f5ffc5441037f41c,Tartu,CE,NaN,Ametikoolitus OÜ,NaN,c68c6310b2fe1124,EI_ILMUNUD_KOHALE,NaN,EI_ILMUNUD,NaN,NaN,2022,8
84342,dabed6dee17e82bc,Pärnu,BE,NaN,osaühing Aide Autokool,NaN,c68c6310b2fe1124,EI_ILMUNUD_KOHALE,NaN,EI_ILMUNUD,NaN,NaN,2022,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129108,9379c80592893a52,Tallinn,BE,NaN,Ametikoolitus OÜ,NaN,c68c6310b2fe1124,EI_ILMUNUD_KOHALE,NaN,EI_ILMUNUD,NaN,NaN,2023,2
28433,1fabcc57525f33af,Tartu,AM,79.02,Tugev Partner OÜ,E,ae264be409f2ca9f,MITTE_SOORITATUD,17.0,EI_ILMUNUD,PAIKNEMINE,NaN,2021,12
96969,ee87cd7c7e0a4368,Valga,B,101,OÜ AUTOSÕIT,E,b89f5e763b05e7a7,SOORITATUD,41.0,EI_ILMUNUD,NaN,NaN,2023,3
91964,7d845201cd5e0b1d,Tallinn,B,101,Sisekaitseakadeemia,E,5c9ee299be483144,SOORITATUD,47.0,EI_ILMUNUD,NaN,NaN,2023,1


In [29]:
combined_data[(combined_data['KATK_POHJUS']=='EI_ILMUNUD')|(combined_data['SEISUND']=='EI_ILMUNUD_KOHALE')]['KATK_POHJUS'].value_counts(dropna=False)

EI_ILMUNUD    2532
TERVIS          49
NaN             27
Name: KATK_POHJUS, dtype: int64

In [ ]:
#drop kõik kes ei ilmunud kohale seisundi järgi, muuda katkestuse põhjus naniks 

In [76]:
combined_data[(combined_data['SEISUND'] == 'MITTE_SOORITANUD') & pd.isna(combined_data['KATK_POHJUS'])]

,EKSAMI_SOORITAJA,BYROO,KATEGOORIA,ERITINGIMUSED,VIIMANE_AUTOKOOL,SOIDUOPETAJA_KAASAS,EKSAMINEERIJA,SEISUND,KESTUS,KATK_POHJUS,MITTEARVESTATUD,VEAD,AASTA,KUU


In [22]:
combined_data[(combined_data['SEISUND'] == 'MITTE_SOORITATUD') & pd.isna(combined_data['MITTEARVESTATUD'])]

,EKSAMI_SOORITAJA,BYROO,KATEGOORIA,ERITINGIMUSED,VIIMANE_AUTOKOOL,SOIDUOPETAJA_KAASAS,EKSAMINEERIJA,SEISUND,KESTUS,KATK_POHJUS,MITTEARVESTATUD,VEAD,AASTA,KUU
394,f608507c3333fc80,Tallinn,B,101,OÜ AUTOSÕIT,E,2bf951502cbf7e6b,MITTE_SOORITATUD,14.0,NaN,NaN,NaN,2021,1
1027,c4cabf33a4b5e5bf,Jõhvi,B,101|78,OÜ AUTOSÕIT,E,d67d168bc8d04278,MITTE_SOORITATUD,32.0,NaN,NaN,NaN,2021,2
2056,8366542874a350a3,Rakvere,B,101,Mittetulundusühing Viljandi Autom,E,147b5bf6bba0d0f1,MITTE_SOORITATUD,3.0,NaN,NaN,NaN,2021,1
5805,7a402e3f327421a2,Haapsalu,B,101,Õnneleid Liikluskool OÜ,E,c347692cf48342d6,MITTE_SOORITATUD,7.0,NaN,NaN,NaN,2021,2
7953,50c9a46bfe7e0cd8,Rakvere,B,101,Osaühing TAPA AUTOKOOL,E,147b5bf6bba0d0f1,MITTE_SOORITATUD,53.0,NaN,NaN,NaN,2021,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115182,e4c18d30ee517a67,Narva,B,101|78,Osaühing STALIKA-VEOD,E,d67d168bc8d04278,MITTE_SOORITATUD,14.0,NaN,NaN,NaN,2023,10
116107,901ba70afaa759f4,Tallinn,B,101,Autom Tallinna Autokool OÜ,E,be39b89bc70a335e,MITTE_SOORITATUD,24.0,NaN,NaN,NaN,2023,10
118091,598bcab771153ab0,Tallinn,B,101|78,OÜ Viktorija Autokool,E,5c9ee299be483144,MITTE_SOORITATUD,15.0,NaN,NaN,NaN,2023,10
119947,99f28768c3564909,Rakvere,B,NaN,NaN,E,147b5bf6bba0d0f1,MITTE_SOORITATUD,34.0,NaN,NaN,NaN,2023,8


In [77]:
combined_copy = combined_copy.drop(columns=['KATK_POHJUS','MITTEARVESTATUD','VEAD'])

In [78]:
combined_copy = combined_copy[(combined_copy['SEISUND'] != 'KATKESTATUD') & (combined_copy['SEISUND'] != 'EI_ILMUNUD_KOHALE')]

In [79]:
combined_copy[pd.isna(combined_copy['KESTUS'])]

,EKSAMI_SOORITAJA,BYROO,KATEGOORIA,ERITINGIMUSED,VIIMANE_AUTOKOOL,SOIDUOPETAJA_KAASAS,EKSAMINEERIJA,SEISUND,KESTUS,AASTA,KUU
18662,9fd723ea6bce84d3,Tartu,AM,79.01,OÜ AUTOSÕIT,0.0,ae264be409f2ca9f,1,NaN,2021,8


In [80]:
combined_copy = combined_copy.drop(18662)

In [111]:
combined_copy_järjestatud = combined_copy.sort_values(by=['EKSAMI_SOORITAJA', 'AASTA', 'KUU','SEISUND'])
combined_copy['VARASEMAID_KATSEID'] = combined_copy_järjestatud.groupby(['EKSAMI_SOORITAJA','KATEGOORIA']).cumcount()

In [112]:
combined_copy[combined_copy['EKSAMI_SOORITAJA']=='7da809d9d7b050c3'].sort_values(by=['AASTA','KUU'])

,EKSAMI_SOORITAJA,BYROO,KATEGOORIA,ERITINGIMUSED,VIIMANE_AUTOKOOL,SOIDUOPETAJA_KAASAS,EKSAMINEERIJA,SEISUND,KESTUS,AASTA,KUU,VARASEMAID_KATSEID
5464,7da809d9d7b050c3,Tallinn,B,101,Stalker Nord Osaühing,0,b353c311e671711a,0,9.0,2021,3,0
10983,7da809d9d7b050c3,Tallinn,B,101,Stalker Nord Osaühing,0,a1d9e404aeec83d1,0,44.0,2021,6,1
17789,7da809d9d7b050c3,Tallinn,B,101,Stalker Nord Osaühing,0,a1d9e404aeec83d1,0,5.0,2021,9,2
58188,7da809d9d7b050c3,Tallinn,B,101,Stalker Nord Osaühing,0,9f3e3d07ecb6a86c,0,12.0,2022,6,3
68030,7da809d9d7b050c3,Tallinn,B,101,Stalker Nord Osaühing,0,f6097462dafbd97c,0,10.0,2022,10,4
72472,7da809d9d7b050c3,Tallinn,B,101,Stalker Nord Osaühing,0,400e86ed874ccd07,0,16.0,2022,11,5
76211,7da809d9d7b050c3,Tallinn,B,101,Stalker Nord Osaühing,0,d15d81b55baf64ed,0,21.0,2022,12,6
91829,7da809d9d7b050c3,Tallinn,B,101,Stalker Nord Osaühing,0,400e86ed874ccd07,0,33.0,2023,1,7
93648,7da809d9d7b050c3,Tallinn,B,101,Stalker Nord Osaühing,0,a1d9e404aeec83d1,0,16.0,2023,1,8
93971,7da809d9d7b050c3,Tallinn,B,101,Stalker Nord Osaühing,0,a1d9e404aeec83d1,0,34.0,2023,2,9


In [113]:
combined_copy = combined_copy.drop(columns='EKSAMI_SOORITAJA')

In [114]:
combined_copy

,BYROO,KATEGOORIA,ERITINGIMUSED,VIIMANE_AUTOKOOL,SOIDUOPETAJA_KAASAS,EKSAMINEERIJA,SEISUND,KESTUS,AASTA,KUU,VARASEMAID_KATSEID
0,Haapsalu,B,101,osaühing KLIMBERG,0,31451a368e962058,1,50.0,2021,1,0
1,Jõgeva,B,101,OÜ AUTOSÕIT,0,ae264be409f2ca9f,0,24.0,2021,1,0
2,Põlva,B,101,H&M MERTENS OÜ,0,40bcf2cea820fca6,0,24.0,2021,1,0
3,Võru,B,101,Tugev Partner OÜ,0,bf2b7218d36e2a3d,0,48.0,2021,1,0
4,Haapsalu,B,101,OÜ Raak Company,0,31451a368e962058,1,51.0,2021,1,0
...,...,...,...,...,...,...,...,...,...,...,...
129212,Viljandi,T,NaN,NaN,0,d321254aa86edd96,1,44.0,2023,4,0
129213,Tallinn,B,NaN,NaN,0,400e86ed874ccd07,1,40.0,2023,5,0
129214,Viljandi,T,NaN,Olustvere Teenindus- ja Maamajanduskool,1,d321254aa86edd96,1,52.0,2023,2,0
129215,Narva,CE,NaN,OSAÜHING ERMAN AUTOKOOL,0,c74b0de43bcd0b0f,0,32.0,2023,10,1


## Visualisation and correlation